In [87]:
#We use VOC2012 dataset, you should download it, and put it in ../data.
#And you also need create a folder called 'labels'
DATA_PATH = "../data/VOC2012/"

In [88]:
#First, we should transform the dataset.
#We need the data type of (cls, x, y, w, h)
CLASSES = ['person', 'bird', 'cat', 'cow', 'dog', 'horse', 'sheep',
           'aeroplane', 'bicycle', 'boat', 'bus', 'car', 'motorbike', 'train',
           'bottle', 'chair', 'dining table', 'potted plant', 'sofa', 'tvmonitor']

In [89]:
import xml.etree.ElementTree as ET
import os
import cv2

def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    
    x = x * dw
    y = y * dh
    w = w * dw
    h = h * dh
    
    return (x, y, w ,h)

def convert_annotation(image_id):
    in_file = open(DATA_PATH + 'Annotations/%s' % image_id)
    image_id = image_id.split('.')[0]
    out_file = open('./labels/%s.txt' % image_id, 'w')
    
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    
    w = int(size.find('width').text)
    h = int(size.find('height').text)
    
    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        
        if cls not in CLASSES or int(difficult) == 1:
            continue
        
        cls_id = CLASSES.index(cls)
        xmlbox = obj.find('bndbox')
        points = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text),
             float(xmlbox.find('ymax').text))
        convert_value = convert((w, h), points)
        
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in convert_value]) + '\n')

def make_label_txt():
    filenames = os.listdir(DATA_PATH + 'Annotations')
    for file in filenames:
        convert_annotation(file)

In [90]:
#Checkout the convert function
def show_labels_img(img_name):
    img = cv2.imread(DATA_PATH + 'JPEGImages/' + img_name + '.jpg')
    h, w = img.shape[:2]
    label = []
    
    with open('./labels/' + img_name + '.txt', 'r') as labels:
        for label in labels:
            label = label.split(' ')
            label = [float(x.strip()) for x in label]
            print(CLASSES[int(label[0])])
            pt1 = (int(label[1] * w - label[3] * w / 2), int(label[2] * h - label[4] * h / 2))
            pt2 = (int(label[1] * w + label[3] * w / 2), int(label[2] * h + label[4] * h / 2))
            cv2.putText(img, CLASSES[int(label[0])], pt1, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255))
            cv2.rectangle(img,pt1,pt2,(0,0,255,2))
    
    cv2.imshow('img', img)
    cv2.waitKey(0)

make_label_txt()
show_labels_img('2007_000027')

person


In [81]:
#Prepare Pytorch Dataset.
class VOC2012(Dataset):
    def __init__(self, is_train=True, is_aug=True):

True
